In [1]:
import os
import pandas as pd 
from bs4 import BeautifulSoup
from lxml import etree
from os import listdir
from os.path import isfile, join

In [2]:
df = pd.DataFrame()
df_lost_time = pd.DataFrame()
print("this is the main df")
print(df)
print("this is the df for lost time")
print(df_lost_time)

this is the main df
Empty DataFrame
Columns: []
Index: []
this is the df for lost time
Empty DataFrame
Columns: []
Index: []


In [3]:
def extractdata(df, filename):

    with open(filename, 'r') as f:

        contents = f.read()
        #this is lxml method to read
        #soup = BeautifulSoup(contents, 'lxml')
        #this is beautiful soup html parser 
        soup = BeautifulSoup(contents, "html.parser")
        dom = etree.HTML(str(soup))

        print("File: " + soup.title.text)

        #assign soup items in to table and rows rows is table body and table rows 
        table = soup.find("table", {"id": "AutoNumber2"})     
        #printing the table
        print(table)
        rows = table.find("tbody").find_all("tr")
        neg_length_rows = 0- len(rows)

        for i in range(-1,neg_length_rows):
            table_data = rows[i].find_all("td")
            table_data[-1].get_text()
            if table_data == "LT Summary":
                break
            else:
                #iterating over the rows from table
                #this will grab all the columns for each row 
                table_data = i.find_all("td")
                length_td = len(table_data)
                print("the lenth of td is:" + table_data)
                #the number of columns per each row is
                # 18 for upper table
                # 11 for lost time table
                # there is an item with 23 elements it is in the middle
                
                #print(getTextFromTag(columns, 18))
                #print(getTextFromTag(columns, lastrow_index)

                if length_td == 11:
                    #This is to assign header data for df
                    #This will be a common data for rows or items in the data frame
                    try:
                        date = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[2]/table/tbody/tr[3]/td')[0].text
                        wellno = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[3]/table[1]/tbody/tr[2]/td')[0].text
                        rigname = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[4]/table/tbody/tr[2]/td')[0].text
                        Date_and_time = table_data[-11].get_text()
                        Time_in_Hrs = table_data[-10].get_text()
                        cum_Hrs = table_data[-9].get_text()
                        LT_ID = table_data[-8].get_text()
                        Parent_LT_ID = table_data[-7].get_text()
                        LT_type = table_data[-6].get_text()
                        cause = table_data[-5].get_text()      
                        object = table_data[-4].get_text()
                        company = table_data[-3].get_text()
                        depth = table_data[-2].get_text()
                        LT_summary = table_data[-1].get_text()
                    except:
                        date = ""
                        wellno = ""
                        rigname = ""
                        Date_and_time = ""
                        Time_in_Hrs = ""
                        cum_Hrs = ""
                        LT_ID = ""
                        Parent_LT_ID = ""
                        LT_type = ""
                        cause = ""      
                        object = ""
                        company = ""
                        depth = ""
                        LT_summary = ""
                    #checking for extra line of information about the depth 
                    #and company this will ignore that line. it is un necessary 
                    # if company == "Depth":
                    #     continue
                        #LT_depth == company
                    data = {
                        #this is is to capture lost time
                        "Report Date": date,
                        "Rig Name": rigname,
                        "Well No": wellno, 
                        "Date and Time": Date_and_time,
                        "Time in Hrs": Time_in_Hrs,
                        "Cummulative Hrs" : cum_Hrs,
                        "LT ID": LT_ID,
                        "Parent LT ID":Parent_LT_ID,
                        "LT Type": LT_type,
                        "Cause" : cause,
                        "Object": object,
                        "Company Name": company, 
                        "Depth": depth,
                        "LT Summary": LT_summary,
                        }
                    
                    df = df.append(data, ignore_index = True)
                else:
                    continue
        return df

In [4]:
#this takes in df and morning reports folder
#it goes through all files in the folder 
#returns data frame 
def readReportsPerDay(df, morningReportFolder):

    mypath = morningReportFolder + "/reports"
    #mypath += "/reports"
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

    count = 1

    for filename in onlyfiles:

        if "DS_Store" in filename:
            continue
        filename = mypath + "/" + filename
        print(str(count) + " " + filename)

        df = extractdata(df, filename)
        count += 1

    return df

In [5]:
#this funtion will call the reports per day function 
#this is to be used for running the program in folder
#it goes thru the all morning report folders per day inside specific folder 
def runInRunningFolder(df):

    for morningReportFolder in listdir(os.getcwd()):

        if ".ipynb" in morningReportFolder or ".git" in morningReportFolder or ".csv" in morningReportFolder or "DS_Store" in morningReportFolder or ".xlsx" in morningReportFolder or ".json" in morningReportFolder or ".pbix" in morningReportFolder or ".zip" in morningReportFolder:
            continue

        print(morningReportFolder + "\n")

        df = readReportsPerDay(df, morningReportFolder)

    return df

In [6]:
print(df)

Empty DataFrame
Columns: []
Index: []


In [7]:
#this is where we run the code for each item in directory 
#this will list curent working directory and rung th code 

print(os.getcwd())


df = runInRunningFolder(df)

print(df)

c:\Users\ygebremedhin\OneDrive - Schlumberger\GitHub\Lost_Time_table_data_Akoji
MR 08-30-2022

1 MR 08-30-2022/reports/088TE-QTIF-519.html
File: 088TE
<table border="0" bordercolor="#111111" cellpadding="2" datafldx="DRLG_OP_SMRY" datapagesizex="5" datasrcx="#data" id="AutoNumber2" style="BORDER-COLLAPSE: collapse; " width="100%">
<thead>
<tr>
<td align="middle" class="cell1h" rowspan="2"> Hrs</td>
<td align="right" class="cell2" rowspan="2"> (From</td>
<td class="cell1h" rowspan="2"> -</td>
<td class="cell1h" rowspan="2" s="">To)</td>
<td align="middle" class="cell2" nowrap="" rowspan="2" width="0%"> Lateral</td>
<td align="middle" class="cell2" nowrap="" rowspan="2" width="0%"> Hole size</td>
<td align="middle" class="cell2" nowrap="" rowspan="2" width="0%"> Cat.</td>
<td align="left" class="cell2" nowrap="" rowspan="2" width="0%"> Major OP</td>
<td align="middle" class="cell2" nowrap="" rowspan="2" width="0%"> Action</td>
<td align="middle" class="cell2" nowrap="" rowspan="2" width=

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Out of scope files/reports'